# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

# Set random seed
np.random.seed(17)

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [3]:
# number of callbacks for black-sounding names
sum(data[data.race=='b'].call)

157.0

In [4]:
data.head()
#data.info()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


<div class="span5 alert alert-success">
<p>Your answers to Q1 and Q2 here</p>
</div>

### Q1. What test is appropriate for this problem? Does CLT apply?
This is a test of two proportions from independent samples. Because there are only two possible outcomes per observation (callback vs. no callback), the data here follows a binomial (Bernoulli distribution). If the following requirements are met, the distribution of sample means from this population can be approximated by a normal distribution, and the CLT will apply. The CLT requirements are (see below for calculations):
1. We have a sufficient number of successes and failures (check: np(1-p) >= 10)
  - For 'white' applications, np(1-p) = 212 (yes, passes)
  - For 'black' applications, np(1-p) = 147 (yes, passes)
2. Sample size is no more than 5% of the population size
  - n_samp = 2435 (for both samples)
  - N_pop = really large (yes, passes)
  
The CLT does apply to this data.

In [5]:
w = data[data.race=='w']
b = data[data.race=='b']

# calculate callback rate for each sample
p_call_w = sum(w.call) / len(w.call)
p_call_b = sum(b.call) / len(b.call)
print('p_call_w =', round(p_call_w, 2), ', p_call_b =', round(p_call_b, 2))

# print success/failure check
print('np(1-p) for "white" applications: ', round(len(w.call) * p_call_w * (1 - p_call_w), 0)) 
print('np(1-p) for "black" applications: ', round(len(b.call) * p_call_b * (1 - p_call_b), 0)) 

p_call_w = 0.1 , p_call_b = 0.06
np(1-p) for "white" applications:  212.0
np(1-p) for "black" applications:  147.0


### Q2. What are the null and alternate hypotheses?
The null and alternate hypotheses are:
- H0: The callback percentage is not different between 'white-sounding' resumes and 'black-sounding' resumes
- H1: The callback percentage is different for 'white-sounding' compared to 'black-sounding' resumes

or:

- H0: p_call_w = p_call_b (equivalent to p_call_w - p_call_b = 0)
- H1: p_call_w != p_call_b (equivalent to p_call_w - p_call_b != 0)

<div class="span5 alert alert-success">
<p>Your solution to Q3 here (Compute margin of error, confidence interval, and p-value)</p>
</div>

### Q3. Compute margin of error, confidence interval, and p-value

Using a frequentist approach (see calculations below), we see that if we set the p-value to 0.05 (two-tailed), the margin of error is 0.015. The observed difference between the two proportions is 0.032, so the 95% confidence interval around the difference is 0.017 to 0.047.

Because the null hypothesis is that the difference between the two proportions is zero and the 95% confidence interval does *not* contain zero, there is sufficient evidence at the p = 0.05 level of significance that 'white-sounding' resumes receive a higher rate of callbacks than 'black-sounding' resumes.

In [6]:
# alpha is 0.05, so calculate z(0.025)
stats.norm.ppf(.025)

# calculate Mean Error
ME = -1*stats.norm.ppf(.025) * np.sqrt((p_call_w*(1-p_call_w)/len(w)) + (p_call_b*(1-p_call_b)/len(b)))
print('Difference between proportions:', round((p_call_w - p_call_b),3))
print('ME: ', round(ME, 3))

# Confidence interval
print('Lower Bound: ', round((p_call_w - p_call_b) - ME, 3), \
     ' Upper Bound: ',round((p_call_w - p_call_b) + ME, 3))

# This interval does not include zero, so it is plausible to think that more 'white' applicants
# got callbacks than 'black' applicants (the p-value was set at p < 0.05)

Difference between proportions: 0.032
ME:  0.015
Lower Bound:  0.017  Upper Bound:  0.047


The bootstrapping approach (see below) uses the following steps:

- Create two arrays which are the observations of calls (True) and no-calls (False) in the 'white-sounding' and 'black-sounding' samples
- Generate many (10,000) replicates by
  - Concatenating the two arrays, permuting, breaking up the permuted array into two arrays with the original sizes
  - Calculating the test statistic (in this case, the proportion of 'black-sounding' applicants that got a callback) in each replicate
- Calculate the probability of getting a proportion of callbacks at least as small as that of the observed proportion of callbacks for the original 'black-sounding' applicant sample

The probability of observing observing a callback rate of 0.06 for the 'black' applicants if there were no real difference between 'white' and 'black' applicants is < 0.01. Therefore it is reasonable to conclude that the callback rates for 'white' and 'black' applicants in this study are not equal.

In [7]:
# 'b' ratio = 157 yes / 2278 no; 'w' ratio = 235 yes / 2200 no
w_applicants = np.array([True] * 235 + [False] * 2200)
b_applicants = np.array([True] * 157 + [False] * 2278)

## These are functions used for bootstrap tests ##
def permutation_sample(data1, data2):
    """Generate a permutation sample from two data sets."""
    
    # Concatenate the data sets: data
    data = np.concatenate((data1, data2))
    
    # Permute the concatenated array: permuted_data
    permuted_data = np.random.permutation(data)
    
    # Split the permuted array into two: perm_sample_1, perm_sample_2
    perm_sample_1 = permuted_data[:len(data1)]
    perm_sample_2 = permuted_data[len(data1):]
    
    return perm_sample_1, perm_sample_2

def draw_perm_reps(data_1, data_2, func, size=1):
    """Generate multiple permutation replicates."""
    
    # Initialize array of replicates: perm_replicates
    perm_replicates = np.empty(size)
    
    for i in range(size):
        # Generate permutation sample
        perm_sample_1, perm_sample_2 = permutation_sample(data_1, data_2)
        
        # Compute the test statistic
        perm_replicates[i] = func(perm_sample_1, perm_sample_2)
        
    return perm_replicates

## This is the function that will be used to generate ##
## the statistic from within the bootstrap tests      ##
def frac_call_blacks(b_applicants, w_applicants):
    "Calculate fraction of callbacks to black applicants."""
    frac = sum(b_applicants) / len(b_applicants)
    return frac

# Draw many permutation samples: perm_replicates
perm_replicates = draw_perm_reps(w_applicants, b_applicants, frac_call_blacks, size=10000)

# Compute and print p-value: p
p = np.sum(perm_replicates <= 157/2278) / len(perm_replicates)
print('p-value = ',p)

p-value =  0.0014


<div class="span5 alert alert-success">
<p> Your answers to Q4 and Q5 here </p>
</div>

### Q4. Write a story describing the statistical significance in the context of the original problem.

In the data from the Bertrand & Mullainathan study, potential employers were sent resumes from (fictional) applicants with black- and white-sounding names, and the researchers recorded whether or not the applicant was selected for an interview. Whereas 9.6% of 'white' applicants were selected for an interview, only 6.4% of the 'black' applicants were selected. Both the bootstrap and frequentist analyses above indicate that this difference is real, and unlikely to have occurred by chance. Because the researchers controlled for other factors (by sending both high and low quality resumes from both 'black' and 'white' applicants to each employer), it is plausible that it is the applicant's perceived race itself that directly impacts the callback rate. If this study is an accurate representation of how  employers truly behave when selecting potential job candidates, this result shows that there is a tangible cost to having a 'black' sounding name. Three fewer black applicants per hundred are selected for interviews than white candidates. 

### Q5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

While race (inferred from applicant name) appears to be a factor in callback success, this analysis does not establish that race is the only or most important factor. The Betrand & Mullainathan study involved sending 'high quality' and 'low quality' resumes to potential employers, and presumably the factors that contributed to a 'high quality' resume should also have been important to callback success. In fact, doing a median split on an individual factor and then calculating the callback proportion for the high and low subgroups (not shown) indicates that higher callback rates may be associated with more years of experience, with higher 'occupation specific' scores, and with higher 'occupation-broad' scores (I did not actually test whether these varying scores significantly affected callback rates).

I think it is likely that race interacts with other factors to affect callback rate. A full analysis would identify other influential factors and then examine whether race modifies their effect on callback rate.